### Knock91: 単一 Data の読み込みをしよう

In [1]:
# 単一 Data の読み込み
import os
import pandas as pd

data_dir = 'data'
store_monthly_dir = os.path.join(data_dir, '01_store_monthly')
ml_base_dir = os.path.join(data_dir, '02_ml_base')

output_ml_result_dir = os.path.join(data_dir, '10_output_ml_result')
output_report_dir = os.path.join(data_dir, '11_output_report')

store_monthly_file = 'store_monthly_data.csv'
ml_base_file = 'ml_base_data.csv'

store_monthly_data = pd.read_csv(os.path.join(store_monthly_dir, store_monthly_file))
ml_base_data = pd.read_csv(os.path.join(ml_base_dir, ml_base_file))
display(store_monthly_data.head(3))
display(ml_base_data.head(3))

,store_name,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,...,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
0,あきる野店,1147,945,202,841,306,844,303,91,122,...,101,95,107,106,100,108,109,96,34.110053,201904
1,さいたま南店,1504,1217,287,1105,399,1104,400,130,135,...,143,142,137,130,113,140,132,155,35.337716,201904
2,さいたま緑店,1028,847,181,756,272,756,272,95,91,...,95,102,82,90,93,95,95,84,34.291617,201904


,store_name,y_weekday,y_weekend,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,...,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
0,あきる野店,1.0,0.0,1147,945,202,841,306,844,303,...,101,95,107,106,100,108,109,96,34.110053,201904
1,さいたま南店,1.0,1.0,1504,1217,287,1105,399,1104,400,...,143,142,137,130,113,140,132,155,35.337716,201904
2,さいたま緑店,1.0,1.0,1028,847,181,756,272,756,272,...,95,102,82,90,93,95,95,84,34.291617,201904


### Knock92: 更新 Data を読み込んで店舗別 Data を作成しよう

In [2]:
# 精度評価結果の読み込み
ml_results_dirs = os.listdir(output_ml_result_dir)
score_all = []
for ml_results_dir in ml_results_dirs:
    score_file_path = os.path.join(output_ml_result_dir, ml_results_dir, 'score.csv')
    score_monthly = pd.read_csv(score_file_path)
    score_monthly['dirs'] = ml_results_dir
    score_all.append(score_monthly)
score_all = pd.concat(score_all, ignore_index=True)
score_all.head()

,DataCategory,acc,f1,recall,precision,tp,fn,fp,tn,model_name,model_target,dirs
0,train,1.000000,1.000000,1.000000,1.000000,861,0,0,777,tree,y_weekday,results_202004
1,test,0.811966,0.829016,0.818414,0.839895,320,71,61,250,tree,y_weekday,results_202004
2,train,1.000000,1.000000,1.000000,1.000000,861,0,0,777,RandomForest,y_weekday,results_202004
3,test,0.792023,0.816121,0.828645,0.803970,324,67,79,232,RandomForest,y_weekday,results_202004
4,train,0.865079,0.875212,0.900116,0.851648,775,86,135,642,GradientBoosting,y_weekday,results_202004


In [3]:
# 年月の抽出
score_all.loc[:, 'year_month'] = score_all['dirs'].str.split('_', expand=True)[1]
score_all.head()

,DataCategory,acc,f1,recall,precision,tp,fn,fp,tn,model_name,model_target,dirs,year_month
0,train,1.000000,1.000000,1.000000,1.000000,861,0,0,777,tree,y_weekday,results_202004,202004
1,test,0.811966,0.829016,0.818414,0.839895,320,71,61,250,tree,y_weekday,results_202004,202004
2,train,1.000000,1.000000,1.000000,1.000000,861,0,0,777,RandomForest,y_weekday,results_202004,202004
3,test,0.792023,0.816121,0.828645,0.803970,324,67,79,232,RandomForest,y_weekday,results_202004,202004
4,train,0.865079,0.875212,0.900116,0.851648,775,86,135,642,GradientBoosting,y_weekday,results_202004,202004


### Knock93: 機械学習 Model の重要変数 Data を読み込んで結合しよう

In [5]:
# 重要度変数 Data の結合
ml_results_dirs = os.listdir(output_ml_result_dir)
importance_all = []
for ml_results_dir in ml_results_dirs:
    importance_file_path = os.path.join(output_ml_result_dir, ml_results_dir, 'importance.csv')
    importance_monthly = pd.read_csv(importance_file_path)
    importance_monthly['dirs'] = ml_results_dir
    importance_all.append(importance_monthly)
importance_all = pd.concat(importance_all, ignore_index=True)
importance_all.loc[:, 'year_month'] = importance_all['dirs'].str.split('_', expand=True)[1]
importance_all.head()

,cols,importance,model_name,model_target,dirs,year_month
0,order_weekend,0.349060,tree,y_weekday,results_202004,202004
1,order_weekday,0.276106,tree,y_weekday,results_202004,202004
2,order,0.050472,tree,y_weekday,results_202004,202004
3,order_takeout,0.034702,tree,y_weekday,results_202004,202004
4,delta_avg,0.030703,tree,y_weekday,results_202004,202004


### Knock94: 機械学習 Model の予測結果を読み込んで結合しよう

In [6]:
# 予測結果 Data の結合
report_files = os.listdir(output_report_dir)
report_all = []
for report_file in report_files:
    report_file_path = os.path.join(output_report_dir, report_file)
    report_monthly = pd.read_excel(report_file_path)
    report_monthly = report_monthly[['store_name', 'score_weekday', 'score_weekend']].copy()
    report_monthly['files'] = report_file
    report_all.append(report_monthly)
report_all = pd.concat(report_all, ignore_index=True)
report_all.head()

,store_name,score_weekday,score_weekend,files
0,あきる野店,0.567821,0.847881,report_pred_202005.xlsx
1,さいたま南店,0.099261,0.775349,report_pred_202005.xlsx
2,さいたま緑店,0.351067,0.898108,report_pred_202005.xlsx
3,さいたま西店,0.256878,0.777520,report_pred_202005.xlsx
4,つくば店,0.158105,0.792846,report_pred_202005.xlsx


In [7]:
# 予測した年月情報の抽出
report_all.loc[:, 'pred_year_month'] = report_all['files'].str.split('.', expand=True)[0]
report_all.loc[:, 'pred_year_month'] = report_all['pred_year_month'].str[12:]
report_all.head()

,store_name,score_weekday,score_weekend,files,pred_year_month
0,あきる野店,0.567821,0.847881,report_pred_202005.xlsx,202005
1,さいたま南店,0.099261,0.775349,report_pred_202005.xlsx,202005
2,さいたま緑店,0.351067,0.898108,report_pred_202005.xlsx,202005
3,さいたま西店,0.256878,0.777520,report_pred_202005.xlsx,202005
4,つくば店,0.158105,0.792846,report_pred_202005.xlsx,202005


### Knock95: 機械学習 Model 用の事前 Data 加工をしよう

In [8]:
ml_data = ml_base_data[['store_name', 'y_weekday', 'y_weekend', 'year_month']].copy()
ml_data.loc[:, 'pred_year_month'] = pd.to_datetime(ml_data['year_month'], format='%Y%m')

from dateutil.relativedelta import relativedelta

ml_data.loc[:, 'pred_year_month'] = ml_data['pred_year_month'].map(lambda x: x + relativedelta(months=1))
ml_data.loc[:, 'pred_year_month'] = ml_data['pred_year_month'].dt.strftime('%Y%m')
del ml_data['year_month']
ml_data.head(3)

,store_name,y_weekday,y_weekend,pred_year_month
0,あきる野店,1.0,0.0,201905
1,さいたま南店,1.0,1.0,201905
2,さいたま緑店,1.0,1.0,201905


In [9]:
# 予測結果 Data と検証 Data の結合
report_valid = pd.merge(report_all, ml_data, on=['store_name', 'pred_year_month'], how='left')
report_valid

,store_name,score_weekday,score_weekend,files,pred_year_month,y_weekday,y_weekend
0,あきる野店,0.567821,0.847881,report_pred_202005.xlsx,202005,0.0,1.0
1,さいたま南店,0.099261,0.775349,report_pred_202005.xlsx,202005,0.0,1.0
2,さいたま緑店,0.351067,0.898108,report_pred_202005.xlsx,202005,0.0,1.0
3,さいたま西店,0.256878,0.777520,report_pred_202005.xlsx,202005,0.0,1.0
4,つくば店,0.158105,0.792846,report_pred_202005.xlsx,202005,0.0,1.0
...,...,...,...,...,...,...,...
970,高津店,0.934106,0.137875,report_pred_202009.xlsx,202009,NaN,NaN
971,高田馬場店,0.965103,0.123156,report_pred_202009.xlsx,202009,NaN,NaN
972,鴻巣店,0.962716,0.145759,report_pred_202009.xlsx,202009,NaN,NaN
973,鶴見店,0.946437,0.153724,report_pred_202009.xlsx,202009,NaN,NaN


In [10]:
report_valid.dropna(inplace=True)
report_valid

,store_name,score_weekday,score_weekend,files,pred_year_month,y_weekday,y_weekend
0,あきる野店,0.567821,0.847881,report_pred_202005.xlsx,202005,0.0,1.0
1,さいたま南店,0.099261,0.775349,report_pred_202005.xlsx,202005,0.0,1.0
2,さいたま緑店,0.351067,0.898108,report_pred_202005.xlsx,202005,0.0,1.0
3,さいたま西店,0.256878,0.777520,report_pred_202005.xlsx,202005,0.0,1.0
4,つくば店,0.158105,0.792846,report_pred_202005.xlsx,202005,0.0,1.0
...,...,...,...,...,...,...,...
775,高津店,0.054634,0.897747,report_pred_202008.xlsx,202008,0.0,1.0
776,高田馬場店,0.112745,0.843095,report_pred_202008.xlsx,202008,0.0,1.0
777,鴻巣店,0.101667,0.896522,report_pred_202008.xlsx,202008,0.0,1.0
778,鶴見店,0.063223,0.935516,report_pred_202008.xlsx,202008,0.0,1.0
